# Zaki 2024 dataset conversion to NWB

This notebook demonstrates how to convert an example session from Zaki 2024 dataset to NWB.

## Notes on the conversion

The conversion notes is located in `src/cai_lab_to_nwb/zaki_2024/notes/zaki_2024_notes.md`. This file contains information about the expected file structure and the conversion process.

## Dataset structure

This notebook demonstrates how to convert an example session to NWB.

This dataset have the following data streams:
* Encoding and Recall sessions:
    - Miniscope imaging (.avi)
    - Video of behavior (.wmv)
    - Freezing analysis output (.csv)
    - Minian Segmentation (.zarr)
    - Shock stimulus times (conversion parameter)
* Offline sessions:
    - Miniscope imaging (.avi)
    - Minian Segmentation (.zarr)
    - EEG and EMG (.edf)
    - Sleep classification (.csv)

## Convert a single session to NWB

The `zaki_2024_convert_session.py` script defines the `session_to_nwb` function that converts a session to NWB.

In [ ]:
from pathlib import Path

### Define conversion parameters
This can be done in three ways:
1. Manually define the parameters for the session you want convert
2. Use the define_conversion_parameters function to populate automatically the conversion_parameter.yaml file 
3. Manually modify the already generated conversion_parameter.yaml file (for example to exclude a specific data stream from the conversion) 

#### 1. Manually define the conversion parameters

In [ ]:
session_to_nwb_kwargs = {
    'date_str': '2022_09_17',
    'edf_file_path': 'D:/Ca_EEG_EDF/Ca_EEG3-4_EDF/Ca_EEG3-4_091722.edf',
    'experiment_dir_path': 'D:/Ca_EEG_Experiment/Ca_EEG3-4/Ca_EEG3-4_Offline/Ca_EEG3-4_OfflineDay1',
    'freezing_output_file_path': None,
    'imaging_folder_path': 'D:/Ca_EEG_Experiment/Ca_EEG3-4/Ca_EEG3-4_Offline/Ca_EEG3-4_OfflineDay1/2022_09_17/17_10_48',
    'minian_folder_path': 'D:/Ca_EEG_Calcium/Ca_EEG3-4/Ca_EEG3-4_OfflineDay1Session16/minian',
    'output_dir_path': 'D:/cai_lab_conversion_nwb',
    'session_description': 'After Neutral Exposure and Fear Conditioning sessions, mice were taken out of the testing chambers and immediately placed in their homecage (scope was not removed).The homecage was placed in a dark grey storage bin with a webcam on top of the bin, taped to a wooden plank, looking down into the homecage. Mouse behavior and calcium were recorded for an hour.',
    'session_id': 'OfflineDay1Session16',
    'shock_stimulus': None,
    'sleep_classification_file_path': 'D:/Ca_EEG_Sleep/Ca_EEG3-4/AlignedSleep/Ca_EEG3-4_OfflineDay1Session16_AlignedSleep.csv',
    'subject_id': 'Ca_EEG3-4',
    'time_str': '17_10_48',
    'video_file_path': None
    }

#### 2. Automatically generate the conversion parameters

In [ ]:
from cai_lab_to_nwb.zaki_2024.utils import update_conversion_parameters_yaml
from neuroconv.utils import load_dict_from_file

subject_id = "Ca_EEG3-4"
session_id = "OfflineDay1Session16"

update_conversion_parameters_yaml(
    subject_id=subject_id,
    data_dir_path=Path("D:/Cai-CN-data-share/"),
    output_dir_path=Path("D:/cai_lab_conversion_nwb/"),
    experiment_design_file_path=Path("D:/Cai-CN-data-share/Ca_EEG_Design.xlsx"),
    session_ids=[session_id],
)

yaml_file_path = "./utils/conversion_parameters.yaml"
conversion_parameter_dict = load_dict_from_file(yaml_file_path)
session_to_nwb_kwargs = conversion_parameter_dict[subject_id][session_id]
session_to_nwb_kwargs

#### 3. Manually modify the conversion parameters from yaml file

In [ ]:
subject_id = "Ca_EEG3-4"
session_id = "OfflineDay1Session16"
yaml_file_path = "./utils/conversion_parameters.yaml"
conversion_parameter_dict = load_dict_from_file(yaml_file_path)
session_to_nwb_kwargs = conversion_parameter_dict[subject_id][session_id]
# For instance if I don't want to include the imaging data in the nwbfile.
print(session_to_nwb_kwargs["imaging_folder_path"],session_to_nwb_kwargs["minian_folder_path"])
session_to_nwb_kwargs.update(
    imaging_folder_path=None,
    minian_folder_path=None,
)
print(session_to_nwb_kwargs["imaging_folder_path"],session_to_nwb_kwargs["minian_folder_path"])

### Run Conversion of a single session
**NB:** Before running the conversion you might want to check that the zaki_2024_metadata.yaml file is reporting all correct information about general session metadata, subject metadata and imaging metadata.

In [ ]:
from cai_lab_to_nwb.zaki_2024.zaki_2024_convert_session import session_to_nwb

print(session_to_nwb.__doc__)

In [ ]:
# Optional parameters
# Whether to overwrite an existing NWB file.
overwrite = True
# Whether to print verbose output.
verbose = True

session_to_nwb(**session_to_nwb_kwargs, overwrite=overwrite, verbose=verbose)

## Convert the week-long session of a subject to NWB

The `zaki_2024_convert_week_session.py` script defines the `session_to_nwb` function that converts a session to NWB.

In [ ]:
from pathlib import Path
from zaki_2024_convert_week_session import session_to_nwb

# Parameters for conversion
data_dir_path = Path("D:/")
output_dir_path = Path("D:/cai_lab_conversion_nwb/")
subject_id = "Ca_EEG3-4"
verbose = True

session_to_nwb(
    data_dir_path=data_dir_path,
    output_dir_path=output_dir_path,
    subject_id=subject_id,
    verbose=verbose
)

## Convert all dataset to nwb
The `zaki_2024_convert_all_sessions.py` script defines the `dataset_to_nwb` function that converts a session to NWB.

**NB: dataset_to_nwb function uses conversion_parameters.yaml to extract automatically the conversion parameters for each session.** 
If you want to run all sessions at once make sure first that the yaml file is populated correctly. `dataset_to_nwb` only convert the sessions defined in the yaml file.
You can auto-populate the yaml file by running `update_conversion_parameters_yaml` function for each subject. 


In [ ]:
from pathlib import Path
from zaki_2024_convert_all_sessions import dataset_to_nwb
# Parameters for conversion
data_dir_path = Path("D:/")
output_dir_path = Path("D:/cai_lab_conversion_nwb/")
max_workers = 1
verbose = True

dataset_to_nwb(
    data_dir_path=data_dir_path,
    output_dir_path=output_dir_path,
    max_workers=max_workers,
    verbose=verbose,
)
